In [1]:
import numpy as np
import gym
import time
import os
from PIL import Image
from scipy.io import savemat
import random

In [2]:
sleep_time = 0.05
path = "data/"

In [3]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [4]:
def game_behavior():
    print("1 - zigzag")
    print("2 - random")
    print("3 - nothing")
    behavior = int(input("Choose behavior of game: "))
    return behavior

In [5]:
behavior = game_behavior()

1 - zigzag
2 - random
3 - nothing
Choose behavior of game: 2


In [6]:
if behavior == 1:
    n_imgs = 120
else:
    n_imgs = 4500

In [7]:
def prepare_folder(PATH):
    m = []
    listdir = os.listdir(PATH)

    if listdir:
        for folder in listdir:
            m.append(int(folder.split("_")[1]))
        number_of_last_folder = max(m)
        next_folder = "match_" + str(number_of_last_folder + 1)
    else:
        next_folder = "match_0"

    match_path = PATH + next_folder
    os.mkdir(match_path)
    
    print(next_folder)
    
    img_path = PATH + next_folder + "/" + "img"
    os.mkdir(img_path)
    
    npz_path = PATH + next_folder + "/" + "npz"
    os.mkdir(npz_path)
    
    mat_path = PATH + next_folder + "/" + "mat"
    os.mkdir(mat_path)
    
    return match_path+"/", img_path+"/", npz_path+"/", mat_path+"/" 

In [8]:
def save_as_png(PATH, pil_buffer):

    print("salva png")

    _path = PATH

    for i in range(len(pil_buffer)):
        
        path = _path + str(i) + ".png"
        pil_buffer[i].save(path)

    print("Successfully saved as PNG.")

In [9]:
def save_as_npz(PATH, match_buffer):
    path = PATH

    np.savez_compressed(path + "frames.npz", match_buffer[1])
    np.savez_compressed(path + "actions.npz", match_buffer[2])
    np.savez_compressed(path + "rewards.npz", match_buffer[3])
    np.savez_compressed(path + "lifes.npz", match_buffer[4])

    print("Successfully saved as NPZ.")

In [10]:
def save_actions_as_txt(PATH, actions):
    f = open(PATH + "actions.txt", "w")
    for i in range(len(actions)):
        f.write(str(actions[i]) + " \n")
    f.close()

In [11]:
def save_as_matfile(PATH, match_buffer):
    mdic = {"num_frames": match_buffer[0], "frames": match_buffer[1], "actions": match_buffer[2]}
    savemat(PATH + "data.mat", mdic)

In [12]:
match_buffer = []
pil_buffer = []
frame_buffer = np.empty((0, y_max-y_min, x_max-x_min), dtype=np.double)
action_buffer = np.empty((0,1), dtype=int) 
reward_buffer = np.empty((0,1), dtype=np.double)
life_buffer = np.empty((0,1), dtype=np.double)

In [13]:
env = gym.make("Enduro-v0")

In [14]:
frame = env.reset()

In [15]:
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

In [16]:
for img in range(n_imgs):
    time.sleep(sleep_time)
    env.render()

    if behavior == 1:
        if img < 30 or img >= 90:
            action = 2
        else:
            action = 3
    elif behavior == 2:
        action = random.randint(1,3)
    else: 
        action = 0

    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")

    pil_buffer.append(frame)

    frame = np.asarray(frame)
    frame = frame.reshape(1, y_max-y_min, x_max-x_min)

    frame_buffer = np.concatenate(( frame_buffer, frame ))
    reward_buffer = np.append(reward_buffer, np.array(reward))
    life_buffer = np.append(life_buffer, np.array(info['ale.lives']))
    action_buffer = np.append(action_buffer, np.array(action))

    frame, reward, done, info = env.step(action)

In [17]:
match_path, img_path, npz_path, mat_path = prepare_folder(path)

match_1003


In [18]:
match_buffer = [action_buffer.shape[0], frame_buffer, action_buffer, reward_buffer, life_buffer]

save_as_npz(npz_path, match_buffer)
save_as_png(img_path, pil_buffer)
save_actions_as_txt(mat_path, action_buffer)
save_as_matfile(mat_path, match_buffer)

Successfully saved as NPZ.
salva png
Successfully saved as PNG.
